# Projeto 1 - Ciência dos Dados

Nome: Rafael Melhado Araujo Lima

Nome: Victor Pacheco Santos

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [174]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    #ACHAR UM JEITO DE RETIRAR OS @S E OS NOMES QUE VEM SEGUIDOS DELES
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed


In [175]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/mac/Desktop/ciencia-dos-dados-p1-main 6


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [176]:
filename = 'iFood.xlsx'

In [177]:
train = pd.read_excel(filename)
train.tail(5)

,Treinamento,Classificação
495,"@rolealeatorio oi, te ajudo com isso lá na dm,...",2
496,@yvnggweed sai fora \nquanto que ta custando t...,2
497,ótima hora pra não funcionar ifood,0
498,@madurezende12 @funkeirosfodas @ifood bem por ...,0
499,"pedi logo um ifood pro almoço, sem coragem pra...",1


In [178]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head()

,Teste
0,@matheusdevdd @rclorandi @rolealeatorio @ifood...
1,@taldoalvesss eu fico parecendo um entregador ...
2,@gdallaru @ifood acordou engraçadinho e com a ...
3,"pedido mais rápido q já fiz no ifood, 20 min f..."
4,"ele não pega ninguém, só leva p dar rolê, ifoo..."


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O produto escolhido foi o serviço do iFood. Para a classificação dos tweets, consideramos relevantes aqueles que forneciam opiniões de clientes, com claras críticas ou elogios. Além disso, classificamos os tweets relevantes como sendo ou positivos ou negativos, para que foçe mais fácil a separação de críticas e elogios.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [179]:
Limpo = []
for i in range (len(train.Treinamento)):
    Limpo.append(cleanup(train.Treinamento[i].lower()))
train["Limpo"] = pd.Series(Limpo)

tudo = ""
for tweet in train.Limpo:
    tudo = tudo + tweet
tudo_separado = tudo.split()

In [180]:
positiva = train.loc[(train.Classificação == 1), "Limpo"]
negativa = train.loc[(train.Classificação == 2), "Limpo"]
irrelevante = train.loc[(train.Classificação == 0), "Limpo"]

positiva_texto = ""
for tweet in positiva:
    positiva_texto = positiva_texto + tweet

negativa_texto = ""
for tweet in negativa:
    negativa_texto = negativa_texto + tweet

    
irrelevante_texto = ""
for tweet in irrelevante:
    irrelevante_texto = irrelevante_texto + tweet
    


positiva_separado = positiva_texto.split()
negativa_separado = negativa_texto.split()
irrelevante_separado = irrelevante_texto.split()


tudo_junto = (positiva_separado+negativa_separado+irrelevante_separado)
prob_p = len(positiva_separado)/len(tudo_separado)
prob_n = len(negativa_separado)/len(tudo_separado)
prob_i = len(irrelevante_separado)/len(tudo_separado)


A celula acima separa os dados do DataFrame baseado na classificação deles (positivo, negativo ou irrelevante) e começa o processo de mudar a formatação desses dados para facilitar o processamento desses dados


In [195]:
serie_positiva = pd.Series(positiva_separado)
tabela_positiva = serie_positiva.value_counts()
tabela_positiva_relativa = serie_positiva.value_counts(True)

serie_irrelevante = pd.Series(irrelevante_separado)
tabela_irrelevante = serie_irrelevante.value_counts()
tabela_irrelevante_relativa = serie_irrelevante.value_counts(True)

serie_negativa = pd.Series(negativa_separado)
tabela_negativa = serie_negativa.value_counts()
tabela_negativa_relativa = serie_negativa.value_counts(True)

serie_junto = pd.Series(tudo_junto)
tabela_junto = serie_junto.value_counts()

Na celula acima, começamos o processo de processamento de dados, criando listas de frequências absolutas e relativas para cada possível classificação


In [192]:
def classifica_tweets(tweet):
    ProbP = prob_p
    ProbN = prob_n
    ProbI = prob_i
    probTweetDadoP = 1
    probTweetDadoN = 1
    probTweetDadoI = 1
    tweet_separado = tweet.split()
    for i in range(len(tweet_separado)):
        if tweet_separado[i] in tabela_positiva_relativa:
            probTweetDadoP = probTweetDadoP*(tabela_positiva[tweet_separado[i]]+1)/(len(serie_positiva) + len(tabela_junto))
        else:
            probTweetDadoP = probTweetDadoP*(0+1)/(len(serie_positiva) + len(tabela_junto))
        if tweet_separado[i] in tabela_negativa_relativa:
            probTweetDadoN = probTweetDadoN*(tabela_negativa[tweet_separado[i]]+1)/(len(serie_negativa) + len(tabela_junto))
        else:
            probTweetDadoN = probTweetDadoN*(0+1)/(len(serie_negativa) + len(tabela_junto))
        if tweet_separado[i] in tabela_irrelevante_relativa:
            probTweetDadoI = probTweetDadoI*(tabela_irrelevante[tweet_separado[i]]+1)/(len(serie_irrelevante) + len(tabela_junto))
        else:
            probTweetDadoI = probTweetDadoI*(0+1)/(len(serie_irrelevante) + len(tabela_junto))
    ProbPDadoTweet = probTweetDadoP*ProbP
    ProbNDadoTweet = probTweetDadoN*ProbN
    ProbIDadoTweet = probTweetDadoI*ProbI
    if ProbPDadoTweet > ProbIDadoTweet and ProbPDadoTweet > ProbNDadoTweet:
        return("O tweet '{0}' é positivo".format(tweet))
    elif ProbNDadoTweet > ProbIDadoTweet and ProbNDadoTweet > ProbPDadoTweet:
        return("O tweet '{0}' é negativo".format(tweet))
    else:
        return("O tweet '{0}' é irrelevante".format(tweet))

Na célula acima, foi criada uma função, que recebe um tweet, e à partir do sistema de classificação de Nayve-Bayes, tomando os dados previamente processados como base, o classifica como positivo, negativo ou irrelevante

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [194]:
teste_limpo = []

for i in range (len(test.Teste)):
    teste_limpo.append(cleanup(test.Teste[i].lower()))
    
classificados = []
for tweet in teste_limpo:
    classificados.append(classifica_tweets(tweet))
    

Quanto à performance do classificador, podemos definir que, dos 200 tweets avaliados:

68 foram corretamente marcados como irrelevantes,

32 foram incorretamente marcados como irrelevantes,

33 foram corretamente marcados como negativos,

23 foram incorretamente marcados como negativos,

18 foram corretamente marcados como positivos,

20 foram incorretamente marcados como positivos.

Com isso, podemos concluir que a acuracia do classificador é de 56,5%.

___
### Concluindo

Em sumo, foi podemos concluir que o classificador funciona de maneira adequada. Porém, ele não é completamente
preciso, tendo uma acuracia de aproximadamente 64%. Além disso, quando analisamos os tweets erroneamente classificados como positivos, vemos a menor precisão. Um dos possíveis motivos para tal discrepância, 
além do tamanho da amostra inicial, é que o classificador não consegue distinguir diferentes contextos,
logo um tweet extremamente sarcástico, pode ser classificado como positivo, quando na verdade é negativo, e 
vice-e-versa, o que exarceba a falta de precisão do classificador. 

Outro possível problema pode ter ocorrido na alimentação de dados ao classificador, sendo que a precisão desse pode  ter sido responsável por problemas no classificador.

Alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets, provavelmente causará grandes problemas, pois o fato de sua acuracia ser relativamente baixa fará com que sejam alimentados muitos dados incorretos, o que propagará vícios do sistema, fazendo com que o classificador se torne cada vez mais impreciso.

Possiveis melhorias para o classificador incluem um aumento na base de dados à ele alimentados e melhores limpezas das bases de dados fornecidas. Além disso, outra possível melhoria para o classificador poderia ser o uso de outro método de classificação, como o metodo de classificação, como o método de máquina de suporte de vetores (https://www.sciencedirect.com/science/article/abs/pii/S0950705108000968)

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**